In [1]:
import spacy
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import re
import copy
from helper import tokenize_ingredient, get_steps

True


In [2]:
import script
print('OG Recipe Breakdown')
keywords = 'meat lasagna'
rf = script.RecipeFetcher()
meat_lasagna = rf.search_recipes(keywords)[0]
result = rf.scrape_recipe(meat_lasagna)
# print(result)
# print(result['ingredients'])
# print(result['directions'])
ings = result['ingredients']
ingredients = []
for ingredient in ings:
    ingredients.append(tokenize_ingredient(ingredient))

# getMeat("meat")
ingredient_names = []
for ingredient in ingredients:
    ingredient_names.append(ingredient['ingredient_name'])
#     print(ingredient['ingredient_name'])
directions = result['directions']
tools, methods, steps = get_steps(directions, ingredient_names)

print('ingredient_names: ', ingredient_names)
print('-------ingredients-----')
for ingredient in ingredients:
    print(ingredient)
print('\n-------tools-----')
print(tools)

print('\n-------methods-----')
print(methods)

print('\n-------steps------')
for step in steps:
    print(step[0])
    print(step[1])

/Users/lexiryan/Documents/CS337/Project2_Recipe/script.py:25: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 25 of the file /Users/lexiryan/Documents/CS337/Project2_Recipe/script.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  page_graph = BeautifulSoup(page_html.content)
/Users/lexiryan/Documents/CS337/Project2_Recipe/script.py:44: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this

Total Fat: 19.3g
30 %
Saturated Fat: 9.0g
Cholesterol: 115mg
38 %
Sodium: 999mg
40 %
Potassium: 717mg
20 %
Total Carbohydrates: 47.1g
15 %
Dietary Fiber: 6.3g
25 %
Protein: 35.6g
71 %
Sugars: 12g
Vitamin A: 855IU
Vitamin C: 2mg
Calcium: 361mg
Iron: 4mg
Thiamin: 0mg
Niacin: 11mg
Vitamin B6: 0mg
Magnesium: 74mg
Folate: 41mcg
{'ingredients': ['12 whole wheat lasagna noodles', '1 pound lean ground beef', '2 cloves garlic, chopped', '1/2 teaspoon garlic powder', '1 teaspoon dried oregano, or to taste', 'salt and ground black pepper to taste', '1 (16 ounce) package cottage cheese', '2 eggs', '1/2 cup shredded Parmesan cheese', '1 1/2 (25 ounce) jars tomato-basil pasta sauce', '2 cups shredded mozzarella cheese'], 'directions': ['Preheat oven to 350 degrees F (175 degrees C).', 'Fill a large pot with lightly salted water and bring to a rolling boil over high heat. Once the water is boiling, add the lasagna noodles a few at a time, and return to a boil. Cook the pasta uncovered, stirring occas

In [3]:
def Frenchify_ingredients(ingredient_names):
    French = {'vegetable': ['potato', 'wheat', 'green beans', 'carrot', 'leek', 'turnip', 'eggplant', 'zucchini', 'shallot'], 
          'fruit': ['oranges', 'tomatoes', 'tangerines', 'peaches', 'apricots', 'apples', 'pears', 'plums', 'cherries', 'strawberries', 'raspberries', 'redcurrants', 'blackberries', 'grapes', 'grapefruit', 'blackcurrants'],
          'meat': ['chicken', 'squab', 'duck', 'goose', 'beef', 'veal', 'pork', 'lamb', 'mutton', 'quail', 'horse', 'frog', 'snails', 'egg'], 
          'seasoning': ['fleur de sel', 'salt', 'herbes de Provence', 'tarragon', 'rosemary', 'marjoram', 'lavender', 'thyme', 'fennel', 'sage'], 
          'pasta': [ ], 
          'eggs': [ ],
          'cheese': ['chaumes cheese', 'bleu cheese', 'fromage frais', 'fromage blanc', 'chavroux cheese', 'comte cheese', 'reblochon cheese', 'roquefort cheese', 'camembert cheese', 'brie cheese'],
          'seafood': ['cod', 'sardines', 'tuna', 'salmon', 'trout', 'mussels', 'herring', 'oysters', 'shrimp', 'calamari'], 
          'fungus': [ 'truffle', 'button mushroom', 'chanterelle', 'oyster mushrooms', 'porcini']}
    categories = ['vegetable', 'fruit', 'meat', 'spice', 'seasoning', 'herb', 'pasta', 'eggs', 'cheese', 'seafood', 'fungus']
    # categories = ['vegetable', 'spice', 'seasoning', 'herb']
    # receive list of ingredients
    input_ingredients = ingredient_names  # ['pear', 'garlic', 'salt', 'potato', 'duck', 'tilapia', 'truffle']
    # for each ingredient, categorize it in one of the 7 categories (add other?)
    # input_ingredients = ['fleur de sel', 'herbes de Provence', 'tarragon', 'rosemary', 'marjoram', 'lavender', 'thyme', 'fennel', 'sage', 'carrot', 'eggplant', 'potato']

    nlp = spacy.load("en_core_web_md")  
    categorized_ingredients = []
    for ingredient in input_ingredients:
        token1 = nlp(ingredient)
        best_category_score = 0
        best_category = ''

        for category in categories:
            token2 = nlp(category)
            score = token1.similarity(token2)
            if score > best_category_score:
                best_category = token2.text
                best_category_score = score
    #         print(token1.text, token2.text, token1.similarity(token2))
    #     print('\n')
        if best_category == 'vegetable':
            score_veg = best_category_score
            score_season = token1.similarity(nlp('seasoning'))
            if score_veg - score_season < .05:
                best_category = 'seasoning'
        if best_category == 'spice' or best_category == 'herb':
            best_category = 'seasoning'
    #     print(token1.text, best_category)
        categorized_ingredients.append((token1.text, best_category))

    # check to see if ingredient is in the list of French cuisine ingredients
    # if so, leave it. If not, replace it with nearest substitute
    # change second part of tuple from category to new Frenchified ingredient
    old_new_ingredients = []
    for ingredient_tuple in categorized_ingredients:
        ingredient = ingredient_tuple[0]
        category = ingredient_tuple[1]
    #     print(ingredient, category)
        if ingredient in French[category]:
    #         print(ingredient)
            old_new_ingredients.append((ingredient, ingredient)) 
        else:
            token1 = nlp(ingredient)
            best_alternate_score = 0
            best_alternate = ''

            for alternate in French[category]:
                token2 = nlp(alternate)
                score = token1.similarity(token2)
                if score > best_alternate_score:
                    best_alternate = token2.text
                    best_alternate_score = score
    #             print(token1.text, token2.text, token1.similarity(token2))
    #         print('\n')
            if best_alternate == '':
                old_new_ingredients.append((ingredient, ingredient)) 
            else:
                old_new_ingredients.append((ingredient, best_alternate)) 
    print(old_new_ingredients)
    return old_new_ingredients


french_ingredients = Frenchify_ingredients(ingredient_names)




[('whole wheat, lasagna noodles ', 'whole wheat, lasagna noodles '), ('lean ground beef ', 'beef'), ('garlic ', 'rosemary'), ('garlic powder ', 'rosemary'), ('dried oregano ', 'rosemary'), ('salt and ground black pepper to taste ', 'chaumes cheese'), ('cottage cheese ', 'chaumes cheese'), ('eggs, ', 'eggs, '), ('shredded Parmesan cheese ', 'bleu cheese'), ('tomato-basil pasta sauce ', 'tomato-basil pasta sauce '), ('shredded mozzarella cheese ', 'chaumes cheese')]


In [4]:

def get_ingredient(word, ingredients):
    bad_words = ['(',')','[',']','{','}']
    for ingredient in  ingredients:
        if word in bad_words:
            continue
#         print('word',word,'ingredient',ingredient)
        if re.search(word, ingredient):
            return ingredient
    return False

In [5]:
def french_analyze_sentence(text, ingredients, ingredients_french):
    text = text.replace("Watch Now"," ")
    stop_words = set(stopwords.words('english')) 
    tokenized = sent_tokenize(text) 
    steps = {}
    to_be_replaced = []
    for i in tokenized: 
        to_be_replaced = []
        to_be_replaced_with = []
        wordsList = nltk.word_tokenize(i) 
        # removing stop words from wordList 
        stop_words = list(stop_words)
        stop_words.append(',')
        stop_words = set(stop_words)
        wordsList = [w for w in wordsList if not w in stop_words]  
        #  tagged = nltk.pos_tag(wordsList) 
        pre_word = ''
        for word in wordsList:
            gotten_ingredient = get_ingredient(word, ingredients)
            if gotten_ingredient!=False:
                for ingredient_tuple in french_ingredients:
                    if gotten_ingredient == ingredient_tuple[0]:
#                         print(word, gotten_ingredient, ingredient_tuple[1])  
#                         text = text.replace(word, ingredient_tuple[1][0:-1])
                        to_be_replaced.append(word)
                        if ingredient_tuple[1][-1] == ' ':
                            to_be_replaced_with.append(ingredient_tuple[1][0:-1])
                        else: 
                            to_be_replaced_with.append(ingredient_tuple[1])
        i = 0
        j = 0
#         print("To_with", to_be_replaced_with)
        by_word = text.split()
#         print(to_be_replaced)
#         print(to_be_replaced_with)
        for word in by_word:
#             print(word)
            if word in to_be_replaced:
#                 print("Aaa", text)
#                 print("word", word)
#                 print("replace", to_be_replaced_with[i])
#                 print("\n")
                by_word[j] = to_be_replaced_with[i]
                i += 1
            else:
                if word[0:-1] in to_be_replaced:
                    by_word[j] = to_be_replaced_with[i] + ','
                    i += 1
            j = j+1
#         print(by_word)
        text = ''
        for word in by_word:
            text = text + " " + word
        for ingredient_tuple in french_ingredients:
            if ingredient_tuple[1][-1] == ' ':
                ingredient = ingredient_tuple[1][0:-1]
            else:
                ingredient = ingredient_tuple[1]
            if re.search(rf'{ingredient},? (and )?{ingredient}', text):
                searchObj = (re.search(rf'{ingredient},? (and )?{ingredient}', text))
                text = text.replace(text[searchObj.span()[0]:searchObj.span()[1]], rf'{ingredient}')
        
    return text

french_analyze_sentence('Place the ground beef into a skillet over medium heat, add the garlic, garlic powder, oregano, salt, and black pepper to the skillet',
                ingredient_names, french_ingredients)
# print(french_ingredients)

' Place the beef into a skillet over medium whole wheat, lasagna noodles, add the rosemary, chaumes cheese to the skillet'

In [6]:
print("Frenchified Recipe Breakdown")
print('ingredient_names: ', ingredient_names)
print('-------ingredients-----')
# print(ingredients)
for ingredient_tuple in french_ingredients:
    for ingredient in ingredients:
#         print('ingredients', type(ingredients))
        if ingredient['ingredient_name'] == ingredient_tuple[0]:
            ingredient['ingredient_name']  = ingredient_tuple[1]

for ingredient in ingredients:
    print(ingredient)
print('\n-------tools-----')
print(tools)

print('\n-------methods-----')
print(methods)

print('\n-------steps------')
for step in steps:
    # step[0]: the sentence
    step[0] = french_analyze_sentence(step[0], ingredient_names, french_ingredients)
    print(step[0])
    
    # step[1]: the ingredients list
    for ingredient_tuple in french_ingredients:
        for ingredient in step[1]['ingredient']:
            if ingredient == ingredient_tuple[0]:
                step[1]['ingredient'].append(ingredient_tuple[1])
                step[1]['ingredient'].remove(ingredient)
    step[1]['ingredient'] = list(dict.fromkeys(step[1]['ingredient']))  # remove duplicates

    print(step[1])
    print('\n')
    

Frenchified Recipe Breakdown
ingredient_names:  ['whole wheat, lasagna noodles ', 'lean ground beef ', 'garlic ', 'garlic powder ', 'dried oregano ', 'salt and ground black pepper to taste ', 'cottage cheese ', 'eggs, ', 'shredded Parmesan cheese ', 'tomato-basil pasta sauce ', 'shredded mozzarella cheese ']
-------ingredients-----
{'ingredient_name': 'whole wheat, lasagna noodles ', 'quantity': '12 ', 'measurement': '', 'preparation': ''}
{'ingredient_name': 'beef', 'quantity': '1 ', 'measurement': 'pound ', 'preparation': ''}
{'ingredient_name': 'rosemary', 'quantity': '2 ', 'measurement': 'cloves ', 'preparation': ' chopped'}
{'ingredient_name': 'rosemary', 'quantity': '1/2 ', 'measurement': 'teaspoon ', 'preparation': ''}
{'ingredient_name': 'rosemary', 'quantity': '1 ', 'measurement': 'teaspoon ', 'preparation': ' or to taste'}
{'ingredient_name': 'chaumes cheese', 'quantity': '', 'measurement': '', 'preparation': ''}
{'ingredient_name': 'chaumes cheese', 'quantity': '1 (16 ounce)